# LangGraph Learning and Testing

This notebook is for learning and experimenting with LangGraph. Let's start by testing our setup and basic imports.

In [2]:
# Test basic imports
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

# Test LangGraph imports
try:
    import langgraph
    print(f"✅ LangGraph version: Installed successfully")
except ImportError as e:
    print(f"❌ LangGraph import failed: {e}")

try:
    import langchain
    print(f"✅ LangChain version: {langchain.__version__}")
except ImportError as e:
    print(f"❌ LangChain import failed: {e}")

try:
    from langchain_openai import ChatOpenAI
    print("✅ LangChain OpenAI imported successfully")
except ImportError as e:
    print(f"❌ LangChain OpenAI import failed: {e}")

Python version: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]
Python executable: /home/homelab/git/master-python/llm-engineering/langgraph-learning/.venv/bin/python
✅ LangGraph version: Installed successfully
✅ LangChain version: 1.0.1
✅ LangChain OpenAI imported successfully


In [ ]:
# Test basic LangGraph functionality
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import operator

# Define a simple state
class State(TypedDict):
    counter: int
    message: str

# Define nodes
def increment_counter(state: State) -> State:
    return {
        "counter": state["counter"] + 1,
        "message": f"Counter incremented to {state['counter'] + 1}"
    }

def double_counter(state: State) -> State:
    new_value = state["counter"] * 2
    return {
        "counter": new_value,
        "message": f"Counter doubled to {new_value}"
    }

# Create a simple graph
workflow = StateGraph(State)
workflow.add_node("increment", increment_counter)
workflow.add_node("double", double_counter)

# Add edges
workflow.add_edge(START, "increment")
workflow.add_edge("increment", "double")
workflow.add_edge("double", END)

# Compile the graph
app = workflow.compile()

print("✅ LangGraph workflow created successfully!")
print("Graph structure:")
print(f"Nodes: {list(app.graph.nodes)}")
print(f"Edges: {list(app.graph.edges)}")

In [ ]:
# Test the workflow execution
initial_state = {
    "counter": 5,
    "message": "Starting with counter = 5"
}

print("Initial state:")
print(f"Counter: {initial_state['counter']}")
print(f"Message: {initial_state['message']}")
print("\n" + "="*50 + "\n")

# Run the workflow
final_state = app.invoke(initial_state)

print("Final state:")
print(f"Counter: {final_state['counter']}")
print(f"Message: {final_state['message']}")
print("\n✅ Workflow executed successfully!")

## Next Steps

Now that we've verified our LangGraph setup is working, we can:

1. **Create more complex workflows** with conditional edges
2. **Integrate with LLMs** using LangChain models
3. **Add memory and persistence** with checkpoints
4. **Build agent-based systems** with tool calling
5. **Create multi-agent workflows** with different roles

The environment is ready for LangGraph learning and experimentation!

In [ ]:
# Environment information
import os
import subprocess

print("📁 Project Directory:", os.getcwd())
print("🐍 Virtual Environment:", os.environ.get('VIRTUAL_ENV', 'Not activated (using UV)'))
print("📦 UV Project:")

# Check if we're in a UV project
if os.path.exists('pyproject.toml'):
    with open('pyproject.toml', 'r') as f:
        content = f.read()
        if '[project]' in content:
            lines = content.split('\n')
            for line in lines:
                if 'name = ' in line:
                    print(f"   Project Name: {line.split('=')[1].strip().strip('"')}")
                elif 'version = ' in line:
                    print(f"   Version: {line.split('=')[1].strip().strip('"')}")
                    break